In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
%matplotlib inline

Using TensorFlow backend.


In [2]:
base_dir = "./data/"

In [3]:
def select_sample(data, n=1):
    np.random.shuffle(data)
    negatives = []
    positives = []
    straights = []
    for d in data:
        if d[1] > 0:
            positives.append(d)
        elif d[1] < 0:
            negatives.append(d)
        else:
            straights.append(d)
    return np.array(negatives[:n] + positives[:n] + straights[:n])

def get_model():
    ch, row, col = 3, 80, 160  # camera format

    model = Sequential()
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same", input_shape=(row, col, ch)))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse", metrics=['mse'])
    return model

def preprocess(img, mean, std):
    img = img.astype("float64")
    img -= mean
    img /= (std + K.epsilon())
    img = cv2.resize(img,None,fx=1/2, fy=1/2, interpolation = cv2.INTER_AREA)
    img = np.expand_dims(img, axis=0)
    return img
    
def data_gen(data, mean, std, batch_size=32, shuffle=True, predict=False):
    i = 0
    n = data.shape[0]
    while 1:
        if shuffle:
            np.random.shuffle(data)
        X, y = get_batch(data, i, batch_size, mean, std)
        i = i+batch_size
        if i > n:
            i = 0
        if predict:
            yield X
        else:
            yield X, y
        
                
def get_batch(data, start_index, batch_size, mean, std):
    n = data.shape[0]
    data_batch = data[start_index:min(start_index+batch_size, n)]
    X = np.zeros((batch_size, 80, 160, 3))
    y = np.zeros(batch_size)
    for i, d in enumerate(data_batch):
        img = plt.imread(base_dir + d[0])
        img = preprocess(img, mean, std)
        X[i,:,:,:] = img
        y[i] = d[1]
    return X, y

            
def normalize(X):
    ##keras code used on a sample of data to get approx of mean and std of image data
    channel_index = 3
    row_index = 1
    col_index = 2
    mean = np.mean(X, axis=(0, row_index, col_index))
    broadcast_shape = [1, 1, 1]
    broadcast_shape[channel_index - 1] = X.shape[channel_index]
    mean = np.reshape(mean, broadcast_shape)
    print("mean: {}".format(mean))
    X -= mean
    
    std = np.std(X, axis=(0, row_index, col_index))
    broadcast_shape = [1, 1, 1]
    broadcast_shape[channel_index - 1] = X.shape[channel_index]
    std = np.reshape(std, broadcast_shape)
    print("std: {}".format(std))
    X /= (std + K.epsilon())
        
def predict_gen(data, mean, std):
    while 1:
        for d in data:
            img = plt.imread(base_dir + d[0])
            img = preprocess(img, mean, std)
            yield img
            
def read_data(array):
    ## used for reading some test data
    array_size = array.shape[0]
    X_train = np.zeros((array_size, 160, 320, 3))
    y_train = np.zeros(array_size)
    print(X_train.shape)
    for i, x in enumerate(array):
        img = plt.imread(base_dir + x[0])
        X_train[i,:,:,:] = img
        y_train[i] = x[1]
    return X_train, y_train 

In [6]:
data = pd.read_csv("./data/driving_log.csv")[['center', 'steering']].values
sample = select_sample(data, n=512)
#X, y = read_data(sample)
#normalize(X)

In [7]:
mean = np.array([[[129.02680093, 133.27253171, 121.95475447]]], dtype='float64')
std = np.array([[[47.02422432, 46.02757431, 57.58701242]]], dtype='float64')

In [8]:
training_data = sample
epochs = 50
validation_data = sample


model = get_model()
model.fit_generator(data_gen(training_data, mean, std, 32), samples_per_epoch=training_data.shape[0],
                    nb_epoch=epochs)

Epoch 1/50
1536/1536 [==============================] - 8s - loss: 0.0580 - mean_squared_error: 0.0580     
Epoch 2/50
1536/1536 [==============================] - 7s - loss: 0.0187 - mean_squared_error: 0.0187     
Epoch 3/50
1536/1536 [==============================] - 7s - loss: 0.0104 - mean_squared_error: 0.0104     
Epoch 4/50
1536/1536 [==============================] - 7s - loss: 0.0088 - mean_squared_error: 0.0088     
Epoch 5/50
1536/1536 [==============================] - 7s - loss: 0.0068 - mean_squared_error: 0.0068     
Epoch 6/50
1536/1536 [==============================] - 7s - loss: 0.0064 - mean_squared_error: 0.0064     
Epoch 7/50
1024/1536 [===================>..........] - ETA: 2s - loss: 0.0064 - mean_squared_error: 0.0064

KeyboardInterrupt: 

In [ ]:
model.predict_generator(data_gen(training_data, mean, std, 3, False, True), training_data.shape[0])

In [ ]:
training_data

In [ ]:
sample